In [1]:
import pandas as pd
import polars as pl

In [2]:
data = "../data/transformed_label_and_damage.parquet"

In [3]:
n_transactions = pl.scan_parquet(data).select(pl.len()).collect()[0,0]
n_transactions_in_test_set = n_transactions * 0.2

In [4]:
# load data from file
df_full = pd.read_csv("../csv/combined_model_regression_on_full_data.csv")
df_diff = pd.read_csv("../csv/combined_model_differences_summary.csv")

# Berechnung des Nutzens des Modells

Um den Nutzen des Modells berechnen zu könenn, müssen die nicht für das Training verwendeten Daten, die per statischen Regelen sicher als FRAUD klassifiziert werden, in die Berechnung eingezogen werden.

Hierzu wird jeweils ein Fünftel der folgenden Kennzahlen verwendet (dies entspricht den 20% Testdaten):
- Schadens 
- Bewertung der statischen Regeln mit Bewertungsfunktion
- Anzahl Transaktionen
verwendet.

In [5]:
damage_static_rules = 5088.38 + 200.7
n_transactions_static_rules = 377 + 16
bewertung_static_rules = n_transactions_static_rules * 5

In [6]:
damage_static_rules_fifth = damage_static_rules * 0.2
n_transactions_static_rules_fifth = n_transactions_static_rules * 0.2
bewertung_static_rules_fifth = bewertung_static_rules * 0.2
print(f"Teil des Schadens in den Daten, die durch statische Regeln erkannt werden: {damage_static_rules_fifth:.2f}")
print(f"Anzahl der Transaktionen {n_transactions_static_rules_fifth}")
print(f"Bewertung der aufgedecketen FRAUD Fälle mit Bewertungsfunktion {bewertung_static_rules_fifth:.2f}")

Teil des Schadens in den Daten, die durch statische Regeln erkannt werden: 1057.82
Anzahl der Transaktionen 78.60000000000001
Bewertung der aufgedecketen FRAUD Fälle mit Bewertungsfunktion 393.00


In [7]:
df_desc = df_full.describe().transpose()

In [8]:
df_desc["mean"]

precision_baseline              0.840331
recall_baseline                 0.552507
recall_combined                 0.529365
precision_combined              0.862492
true_positives_baseline       472.725000
false_positives_baseline       89.925000
true_positives_combined       452.925000
false_positives_combined       72.300000
damage_in_test_set           6015.164000
damage_prevented_baseline    3550.442900
damage_prevented_combined    3468.273300
bewertung_baseline          -1000.346100
bewertung_combined          -1005.265700
difference_bewertung           -4.919600
Name: mean, dtype: float64

In [10]:
damage = df_desc.loc["damage_in_test_set", "mean"]
bewertung_model = df_desc.loc["bewertung_combined", "mean"]
damage_prevented = df_desc.loc["damage_prevented_combined", "mean"]

In [11]:
summary = pd.DataFrame({
    "Schaden": [-damage, -damage_static_rules_fifth],
    "Bewertung": [bewertung_model, bewertung_static_rules_fifth],
    "Mehrwert": [bewertung_model + damage, bewertung_static_rules_fifth + damage_static_rules_fifth],
    "Anzahl Transaktionen": [n_transactions_in_test_set, n_transactions_static_rules_fifth],
})
summary.index = ["Model", "Statische Regeln"]
summary = summary.T
summary["Gesamt"] = summary.sum(axis=1)
summary.T.round(2)


,Schaden,Bewertung,Mehrwert,Anzahl Transaktionen
Model,-6015.16,-1005.27,5009.90,29604.8
Statische Regeln,-1057.82,393.00,1450.82,78.6
Gesamt,-7072.98,-612.27,6460.71,29683.4


In [16]:
mehrwert_pro_transaktion = summary.loc["Mehrwert", "Gesamt"] / summary.loc["Anzahl Transaktionen", "Gesamt"]
print(f"Mehrwert des Modells pro Transaktion: {mehrwert_pro_transaktion:.3f}")

Mehrwert des Modells pro Transaktion: 0.218


In [13]:
summaryd = pd.DataFrame({
    "Schaden verhindert": [damage_prevented, damage_static_rules_fifth],
    "Anzahl Transaktionen": [n_transactions_in_test_set, n_transactions_static_rules_fifth],
})
summaryd.index = ["Model", "Statische Regeln"]
summaryd = summaryd.T
summaryd["Gesamt"] = summaryd.sum(axis=1)
summaryd.T.round(2)


,Schaden verhindert,Anzahl Transaktionen
Model,3468.27,29604.8
Statische Regeln,1057.82,78.6
Gesamt,4526.09,29683.4


In [15]:
verlustminderung_pro_transaktion = summaryd.loc["Schaden verhindert", "Gesamt"] / summaryd.loc["Anzahl Transaktionen", "Gesamt"]
print(f"Verlustminderung des Modells pro Transaktion: {verlustminderung_pro_transaktion:.3f}")


Verlustminderung des Modells pro Transaktion: 0.152
